In [1]:
!pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 3.9MB/s eta 0:00:01


In [1]:
import psycopg2

In [2]:
dir(psycopg2)

['BINARY',
 'Binary',
 'DATETIME',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NUMBER',
 'NotSupportedError',
 'OperationalError',
 'ProgrammingError',
 'ROWID',
 'STRING',
 'Time',
 'TimeFromTicks',
 'Timestamp',
 'TimestampFromTicks',
 'Warning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__libpq_version__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_connect',
 '_ext',
 '_json',
 '_psycopg',
 '_range',
 'apilevel',
 'compat',
 'connect',
 'errors',
 'extensions',
 'paramstyle',
 'threadsafety',
 'tz']

In [3]:
help(psycopg2.connect().cursor().commit())

TypeError: missing dsn and no parameters

In [4]:
dbname = 'lxgfrdqi'
user = 'lxgfrdqi'
password = 'VWf_fCSBuL_cHzWjnfB9U27AmZdZtOto' #don't commit this
host = 'raja.db.elephantsql.com'

In [5]:
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                          password=password, host=host)

In [6]:
pg_conn

<connection object at 0x7f06c60b3b40; dsn: 'user=lxgfrdqi password=xxx dbname=lxgfrdqi host=raja.db.elephantsql.com', closed: 0>

In [7]:
pg_curs = pg_conn.cursor()

In [8]:
pg_curs.execute('SELECT * from test_table;')
pg_curs.fetchall()

# any valid json can be a dictionary, but not vice versa b/c json require keys to be strings

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

In [10]:
# get the rpg data
!wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
!mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3

--2019-06-04 09:47:14--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-06-04 09:47:14--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-06-04 09:47:14--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module

In [40]:
import sqlite3
sl_conn = sqlite3.connect('rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [41]:
characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()

In [42]:
len(characters)

302

In [43]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [53]:
# now characters is in a local python object (list of tuples)
# next create table in postgresql
# make a schema

create_character_table = """
CREATE TABLE charactercreator_character(
    character_id SERIAL PRIMARY KEY,
    name VARCHAR(30),
    level INT,
    exp INT,
    hp INT,
    strength INT,
    intelligence INT,
    dexterity INT,
    wisdom INT
)
"""

In [54]:
pg_curs.execute(create_character_table)

In [55]:
str(characters[0])

"(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [56]:
attempted_insert = """
    INSERT INTO charactercreator_character
    VALUES """+ str(characters[0])
attempted_insert

"\n    INSERT INTO charactercreator_character\n    VALUES (1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [57]:
pg_curs.execute(attempted_insert)

In [58]:
pg_conn.commit()

In [59]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [60]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [61]:
len(pg_characters)

1

In [62]:
# let's insrt the rest of the characters
for character in characters[1:]:
    insert_character="""
    INSERT INTO charactercreator_character
    VALUES """ + str(character)
    pg_curs.execute(insert_character)
pg_conn.commit()

In [63]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [64]:
len(pg_characters)

302

In [65]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [66]:
pg_characters[-1]

(302, 'Aliquam n', 0, 0, 10, 1, 1, 1, 1)

In [67]:
len(characters)

302

In [68]:
for character, pg_character in zip(characters, pg_characters):
    assert character == pg_character

In [69]:
# no error! it worked!